In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_kan
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'kan_train_negative_augmented')
    dev_data = read_task(dataset_location , split = 'kan_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_kan
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Kannada' + 'Augmented' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 0.8509988784790039
DEV ACC: 0.605936004784689
DEV Precision: 0.4284304250722892
DEV Recall: 0.4526405383089875
DEV F1Score: 0.40415154645957513
BEST ACCURACY -->  DEV: 0.60594
BEST PRECISION -->  DEV: 0.42843
BEST RECALL -->  DEV: 0.45264
BEST F1SCORE -->  DEV: 0.40415
TIME PER EPOCH: 5.416618208090465

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIH

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 0.9845826625823975
DEV ACC: 0.5892643540669856
DEV Precision: 0.406347271027994
DEV Recall: 0.4061760869849105
DEV F1Score: 0.3805379703222544
BEST ACCURACY -->  DEV: 0.58926
BEST PRECISION -->  DEV: 0.40635
BEST RECALL -->  DEV: 0.40618
BEST F1SCORE -->  DEV: 0.38054
TIME PER EPOCH: 5.281254529953003

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 0.9861420392990112
DEV ACC: 0.6110944976076556
DEV Precision: 0.4696753439511672
DEV Recall: 0.443209466558397
DEV F1Score: 0.4311640850959922
BEST ACCURACY -->  DEV: 0.61109
BEST PRECISION -->  DEV: 0.46968
BEST RECALL -->  DEV: 0.44321
BEST F1SCORE -->  DEV: 0.43116
TIME PER EPOCH: 5.741048888365428

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.142885684967041
DEV ACC: 0.5813397129186603
DEV Precision: 0.4522714496428612
DEV Recall: 0.4985207494291366
DEV F1Score: 0.4518585365159311
BEST ACCURACY -->  DEV: 0.58134
BEST PRECISION -->  DEV: 0.45227
BEST RECALL -->  DEV: 0.49852
BEST F1SCORE -->  DEV: 0.45186
TIME PER EPOCH: 5.881382123629252

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHE

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 1.1169543266296387
DEV ACC: 0.6093002392344498
DEV Precision: 0.4648324202144918
DEV Recall: 0.472445972500152
DEV F1Score: 0.4550091503535436
BEST ACCURACY -->  DEV: 0.6093
BEST PRECISION -->  DEV: 0.46483
BEST RECALL -->  DEV: 0.47245
BEST F1SCORE -->  DEV: 0.45501
TIME PER EPOCH: 5.569566647211711

Training epoch: 2
FINSIHED BATCH: 19 of 246
FINSIHED BATCH: 39 of 246
FINSIHED BATCH: 59 of 246
FINSIHED BATCH: 79 of 246
FINSIHED BATCH: 99 of 246
FINSIHED BATCH: 119 of 246
FINSIHED BATCH: 139 of 246
FINSIHED BATCH: 159 of 246
FINSIHED BATCH: 179 of 246
FINSIHED BATCH: 199 of 246
FINSIHED BATCH: 219 of 246
FINSIHED BATCH: 239 of 246
Validation loss per 100 evaluation steps: 0.9176064133644104


In [6]:
print(best_dev_acc)

0.6590161483253588


In [7]:
print(best_epoch)

6


In [8]:
print(best_dev_precision)

0.5600124810689826


In [9]:
print(best_dev_recall)

0.5685806506913671


In [10]:
print(best_dev_f1score)

0.5394615213907614


In [11]:
print(overall_list_dev_acc)

[[0.605936004784689, 0.6187200956937798, 0.650941985645933, 0.6552033492822966, 0.6447368421052632, 0.6622308612440192, 0.6518391148325359, 0.6295604066985646, 0.6499700956937798, 0.6110197368421052], [0.5892643540669856, 0.6484748803827751, 0.6578947368421052, 0.6300837320574163, 0.6167763157894737, 0.6498953349282297, 0.6376345693779905, 0.6119168660287081, 0.6067583732057416, 0.6272428229665071], [0.6110944976076556, 0.6214862440191388, 0.6465311004784688, 0.6546800239234449, 0.6594647129186603, 0.6366626794258373, 0.6452601674641149, 0.6380831339712919, 0.6214862440191388, 0.6077302631578948], [0.5813397129186603, 0.6167763157894737, 0.6257476076555024, 0.6489982057416268, 0.6243271531100478, 0.6253738038277512, 0.6313546650717703, 0.6347936602870813, 0.6258223684210527, 0.6177482057416268], [0.6093002392344498, 0.6590161483253588, 0.6513157894736842, 0.6389055023923444, 0.6413726076555024, 0.6418959330143541, 0.6552781100478469, 0.6556519138755981, 0.6357655502392344, 0.6091507177

In [12]:
print(overall_list_dev_precision)

[[0.4284304250722892, 0.49371403848676587, 0.5417466749165754, 0.5361863479345185, 0.5037221823833643, 0.5465377954158869, 0.5351378710298987, 0.5293894950310073, 0.5625409179743546, 0.5140120439884724], [0.406347271027994, 0.5112978664793957, 0.4731796448461942, 0.502719841062087, 0.49402891523212383, 0.5404763357479373, 0.5263027153679419, 0.5108799562307161, 0.5079916282547862, 0.5228226629985706], [0.4696753439511672, 0.47633393924538736, 0.5025030095463533, 0.5586520149488063, 0.5331671557325935, 0.5369448448008818, 0.5330706182050117, 0.5304872705710026, 0.5124569724221426, 0.49907815227896857], [0.4522714496428612, 0.46430953013462906, 0.4607137932743054, 0.4838681120611884, 0.5005766588932223, 0.5046655730359645, 0.5057497461278597, 0.5215993884699908, 0.5272064247398472, 0.5180218993721667], [0.4648324202144918, 0.5409315921620037, 0.539829275588845, 0.5404506618369903, 0.5600124810689826, 0.5491406855567423, 0.559271060818345, 0.541793601760531, 0.5348820876484118, 0.55307969

In [13]:
print(overall_list_dev_recall)

[[0.4526405383089875, 0.53955989986809, 0.5308422076537049, 0.5033722115673988, 0.4917951086815428, 0.49440450528687346, 0.5452067279438516, 0.5344375319695472, 0.5243300937890017, 0.49921262217314855], [0.4061760869849105, 0.5129911660797533, 0.5123214876564159, 0.5763671265173517, 0.5239419037528911, 0.5202806868054547, 0.5144405156637777, 0.5131056453319331, 0.48318099626395433, 0.5046184679301059], [0.443209466558397, 0.48339644980687774, 0.5102820632943065, 0.5038969199171844, 0.504935056773292, 0.5290445053898474, 0.5387148665357221, 0.5339080393759539, 0.5174948792295049, 0.4863205593855749], [0.4985207494291366, 0.4181108009559319, 0.4755245623751112, 0.5049444282068121, 0.48664949935372825, 0.5146320111620561, 0.5157555032823292, 0.5402187510163164, 0.5399946988699451, 0.5175916730328495], [0.472445972500152, 0.5158564603220528, 0.5411590927874701, 0.5398403557227086, 0.5158948931341979, 0.5405540411756988, 0.5556912174395638, 0.5685806506913671, 0.5338091482508173, 0.53333830

In [14]:
print(overall_list_dev_f1score)

[[0.40415154645957513, 0.4866003075855227, 0.5106780346173401, 0.490687670418339, 0.47983111543040696, 0.49291914258712777, 0.5176940491792895, 0.5104532312533221, 0.5168169555901642, 0.48311415796998036], [0.3805379703222544, 0.48381291633575196, 0.47224368084571644, 0.5045644933305501, 0.4848756708097291, 0.5067693951562542, 0.496957854530467, 0.4871949478139234, 0.47894413271941344, 0.4937903420264407], [0.4311640850959922, 0.46580299533840575, 0.48216416681374524, 0.5077530141567711, 0.49300665942422073, 0.5119585705852079, 0.5110409315621925, 0.5172114597388853, 0.49537284257922687, 0.4780318628155863], [0.4518585365159311, 0.4111389806069918, 0.44615528837860086, 0.47996191765943014, 0.4697773092461148, 0.48946563539546806, 0.4807306545562607, 0.5094423059243468, 0.5069652296970177, 0.490808264247982], [0.4550091503535436, 0.4948205542825652, 0.5210658794562405, 0.5211515147147747, 0.5171446724952792, 0.5167410684021957, 0.5394615213907614, 0.5360745021121144, 0.5151950928457564,

In [15]:
#The best model is 0